REQUIREMENTS.TXT

EXAMPLE_MD_TO_TEXT.TXT

example.pdf

HF_TOKEN.txt

GROQ_TOKEN.txt


In [16]:
with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
    markdown_example = f.read()

with open("example.pdf", "rb") as f:
    pdf = f.read()

with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.6 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement pywin32==308 (from versions: none)
ERROR: No matching distribution found for pywin32==308


In [3]:
! python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 68.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

alberto-lorente


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
import torch

device = "cpu"
if torch.cuda.is_available():
  print("Cuda available")
  device = torch.device('cuda')

Cuda available


### INITIAL PREPROCESSING RECAP

So far for the text we have:
- the reconstructed text based on the paragraph clusters.
- the embeddings of the reconstructed clusters.
- the reconstructed paragraphs in ranges of 10 sentences.
- the embeddings of the paragraphs detached and squeezed.
- the cluster each paragraph belongs to.
- the indexes of the paragraphs of each cluster.
- the actual sentences split from SpaCy, in case we need them after.

For the images:
- Index for the pdfs with ColPali that only return pages.
- Querying images with Groq that seems pretty good.


## Post Processing

## Sumarization

Now that we have the basic separation wrt to clusters, we can start creating different representations for each cluster and doing the hierarchical indexing.

In [ ]:
!pip install protobuf


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from transformers import AutoModelForSeq2SeqLM

summ_model = 'csebuetnlp/mT5_multilingual_XLSum' # take the one for mt5 french!!

tokenizer_summ = AutoTokenizer.from_pretrained(summ_model)
model_summ = AutoModelForSeq2SeqLM.from_pretrained(summ_model).to(device)

c:\Users\alber\Desktop\envs\.council-rag\.council-rag\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
import re
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip())) # was part of the docs

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\alber\AppData\Local\Temp\ipykernel_20020\3419883012.py:2: SyntaxWarning: invalid escape sequence '\s'
  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip())) # was part of the docs


### Individual example

In [ ]:
example_cluster = clusters_ids["cluster_0"]["union_paras"]
example_cluster

"PROCES VERBAL\n\n\n\n L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle Vacheresse \x96 Rue des Vernes à Roanne.\n\n\n\n La convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n # Etaient présents :\n\n\n\nChristine Aranéo - Marcel AugierDana Burnichon| | |Edmond Bourgeon| |\n\n|-| | |Catherine Brun| |\n\n|Jean-Marc Detour| | | | |\n\n|Daniel Fréchet| | |Lucien Murzi| |\n\n|Quentin Guillermin|X| | | |\n\n|Jean-Paul Heyberger| | |Christelle Lattat| |\n\n|Franck Maupetit| | |Annie Gerenton| |\n\n|Patrick Meunier|X| | | |\n\n|Nabih Nejjar|X| | |(arrivé en cours de séance)|\n\n|Gilles Passot| | |Fabien Lambert|(arrivé en cours de séance)|\n\n|Marcel Peuillon|X| | | |\n\n|Didier Prunet| | |Christian Dupuis| |\n\n|Vickie Redeuilh| | |Hélène Lapalus| |\n\n|Marie-Hélène Riamon| | |Denis V

In [ ]:
tokenized_example = tokenizer_summ(
                              [WHITESPACE_HANDLER(example_cluster)],
                              return_tensors="pt",
                              padding="max_length",
                              truncation=True
                          ).to(device)

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
tokenized_example

{'input_ids': tensor([[   259, 167667,  40178,  ...,  30770,   1998,      1]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}

In [ ]:
input_ids = tokenized_example["input_ids"]
# input_ids = tokenized_example["input_ids"].squeeze(dim=0) NOP,expects the unsqueexed dimension

In [ ]:
input_ids

In [ ]:
len(input_ids[0])

5677

In [ ]:
torch.cuda.empty_cache()
output = model_summ.generate(
    input_ids=input_ids,
    max_length=300,
    no_repeat_ngram_size=2,
    num_beams=1) # if it is too slow, adjust num_beams and max_len

c:\Users\alber\Desktop\envs\.council-rag\.council-rag\Lib\site-packages\transformers\generation\configuration_utils.py:695: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.6` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


In [ ]:
output[0]

In [ ]:
summary = tokenizer_summ.decode(
    output[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

In [ ]:
summary

### For the clusters

In [ ]:
from torch.amp import autocast

In [ ]:
def generate_summary(concat_text):

  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

  tokenized_example = tokenizer_summ(
                              [WHITESPACE_HANDLER(concat_text)],
                              return_tensors="pt",
                              padding="max_length",
                              truncation=True).to(device)

  input_ids = tokenized_example["input_ids"]

  model_summ.eval()
  with autocast("cuda"):
    summary_embedding_output = model_summ.generate(
                                              input_ids=input_ids,
                                              max_length=1000,
                                              no_repeat_ngram_size=2,
                                              num_beams=4) # if it is too slow, adjust num_beams and max_len

  summary_decoded = tokenizer_summ.decode(
                                    summary_embedding_output[0],
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=False)

  torch.cuda.empty_cache() # Careful with running out of memory

  return summary_embedding_output.detach().cpu().numpy(), summary_decoded

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def process_cluster(cluster, cluster_aggregates=cluster_aggregates):

  print("Procesing cluster ", cluster+1)
  torch.cuda.empty_cache()

  cluster_aggregates[cluster]["broader_representations"] = {}

  concat_text = " ".join(cluster_aggregates[cluster]["texts"])
  cluster_aggregates[cluster]["broader_representations"]["concat_text"] = concat_text


  cluster_embeddings = compute_norm_embeddings(concat_text)
  cluster_aggregates[cluster]["broader_representations"]["cluster_sentence_embeddings"] = cluster_embeddings[0] # taking out the extra unnecesary dimension

  torch.cuda.empty_cache()

  summary_embedding_output, summary_decoded = generate_summary(concat_text)
  cluster_aggregates[cluster]["broader_representations"]["cluster_summary_embedding"] = summary_embedding_output
  cluster_aggregates[cluster]["broader_representations"]["cluster_summary_decoded"] = summary_decoded

  return cluster_aggregates


In [ ]:
for cluster in list(cluster_aggregates.keys()):
  cluster_aggregates = process_cluster(cluster)

In [ ]:
cluster_aggregates[1]["broader_representations"].keys()

In [ ]:
cluster_aggregates[1]["broader_representations"]["cluster_summary_decoded"]

In [ ]:
pprint(cluster_aggregates[1]["broader_representations"]["concat_text"])

In [ ]:
cluster_aggregates[0]["broader_representations"]["cluster_summary_decoded"]